# VideoPrism Video-Text Encoder Demo

[![Paper](https://img.shields.io/badge/arXiv-2402.13217-red.svg)](https://arxiv.org/abs/2402.13217)
[![Blog](https://img.shields.io/badge/Google_Research-Blog-green.svg)](https://research.google/blog/videoprism-a-foundational-visual-encoder-for-video-understanding/)
[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](https://opensource.org/licenses/Apache-2.0)

This notebook provides an example of video and text feature extraction with a pre-trained VideoPrism video-text model for zero-shot video classification/retrieval.

Please run this demo on Google Colab with (faster) or without TPU.

## Set up

In [1]:
# @title Prepare environment

import os
import sys

# Fetch VideoPrism repository if Python does not know about it and install
# dependencies needed for this notebook.
if not os.path.exists("videoprism_repo"):
  !git clone --quiet --branch=main --depth=1 \
     https://github.com/google-deepmind/videoprism.git videoprism_repo
  os.chdir('./videoprism_repo')
  !pip install .
  os.chdir('..')

# Append VideoPrism code to Python import path.
if "videoprism_repo" not in sys.path:
  sys.path.append("videoprism_repo")

# Install missing dependencies.
!pip install mediapy

import jax
from jax.extend import backend
import tensorflow as tf

# Do not let TF use the GPU or TPUs.
tf.config.set_visible_devices([], "GPU")
tf.config.set_visible_devices([], "TPU")

print(f"JAX version:  {jax.__version__}")
print(f"JAX platform: {backend.get_backend().platform}")
print(f"JAX devices:  {jax.device_count()}")

JAX version:  0.5.2
JAX platform: tpu
JAX devices:  8


In [2]:
# @title Load dependencies and define utilities

import mediapy
import numpy as np
from PIL import Image


def read_and_preprocess_video(
    filename: str, target_num_frames: int, target_frame_size: tuple[int, int]
):
  """Reads and preprocesses a video."""

  frames = mediapy.read_video(filename)

  # Sample to target number of frames.
  frame_indices = np.linspace(
      0, len(frames), num=target_num_frames, endpoint=False, dtype=np.int32
  )
  frames = np.array([frames[i] for i in frame_indices])

  # Resize to target size.
  original_height, original_width = frames.shape[-3:-1]
  target_height, target_width = target_frame_size
  assert (
      original_height * target_width == original_width * target_height
  ), 'Currently does not support aspect ratio mismatch.'
  frames = mediapy.resize_video(frames, shape=target_frame_size)

  # Normalize pixel values to [0.0, 1.0].
  frames = mediapy.to_float01(frames)

  return frames


def compute_similarity_matrix(
    video_embeddings,
    text_embeddings,
    temperature: float,
    apply_softmax: str | None = None,
) -> np.ndarray:
  """Computes cosine similarity matrix."""
  assert apply_softmax in [None, 'over_texts', 'over_videos']
  emb_dim = video_embeddings[0].shape[-1]
  assert emb_dim == text_embeddings[0].shape[-1]

  video_embeddings = np.array(video_embeddings).reshape(-1, emb_dim)
  text_embeddings = np.array(text_embeddings).reshape(-1, emb_dim)
  similarity_matrix = np.dot(video_embeddings, text_embeddings.T)

  if temperature is not None:
    similarity_matrix /= temperature

  if apply_softmax == 'over_videos':
    similarity_matrix = np.exp(similarity_matrix)
    similarity_matrix = similarity_matrix / np.sum(
        similarity_matrix, axis=0, keepdims=True
    )
  elif apply_softmax == 'over_texts':
    similarity_matrix = np.exp(similarity_matrix)
    similarity_matrix = similarity_matrix / np.sum(
        similarity_matrix, axis=1, keepdims=True
    )

  return similarity_matrix

In [3]:
# @title Load model

import jax
import jax.numpy as jnp
from videoprism import models as vp

# MODEL_NAME = 'videoprism_lvt_public_v1_base'  # @param ['videoprism_lvt_public_v1_base', 'videoprism_lvt_public_v1_large'] {allow-input: false}
MODEL_NAME = 'videoprism_lvt_public_v1_large'
NUM_FRAMES = 16
FRAME_SIZE = 288

flax_model = vp.get_model(MODEL_NAME)
loaded_state = vp.load_pretrained_weights(MODEL_NAME)
text_tokenizer = vp.load_text_tokenizer('c4_en')


@jax.jit
def forward_fn(inputs, text_token_ids, text_paddings, train=False):
  return flax_model.apply(
      loaded_state,
      inputs,
      text_token_ids,
      text_paddings,
      train=train,
  )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


flax_lvt_large_f8r288_repeated.npz:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

# Example: Zero-shot Video Classification/Retrieval

In this example, we extract the embedding of an input video, and the embeddings of five senetence. We measure the cosine similarites between the videos and sentences.

In [4]:
# @title Specify input video
VIDEO_FILE_PATH = 'videoprism_repo/videoprism/assets/water_bottle_drumming.mp4'  # @param {type: "string"}

frames = read_and_preprocess_video(
    VIDEO_FILE_PATH,
    target_num_frames=NUM_FRAMES,
    target_frame_size=[FRAME_SIZE, FRAME_SIZE],
)
frames = jnp.asarray(frames[None, ...])  # Add batch dimension.

In [5]:
# @title Specify input text queries
TEXT_QUERY_CSV = 'playing drums,sitting,playing flute,playing at playground,concert'  # @param {type: "string"}
PROMPT_TEMPLATE = 'a video of {}.'

text_queries = TEXT_QUERY_CSV.split(',')
text_queries = [PROMPT_TEMPLATE.format(t) for t in text_queries]
text_ids, text_paddings = vp.tokenize_texts(text_tokenizer, text_queries)

print('Input text queries:')
for i, text in enumerate(text_queries):
  print(f'({i + 1}) {text}')

Input text queries:
(1) a video of playing drums.
(2) a video of sitting.
(3) a video of playing flute.
(4) a video of playing at playground.
(5) a video of concert.


In [18]:
# text_ids

In [6]:
# @title Compute video-to-text retrieval results
video_embeddings, text_embeddings, _ = forward_fn(
    frames, text_ids, text_paddings)

TEMPERATURE = 0.01  # @param {type: "number"}
similarity_matrix = compute_similarity_matrix(
    video_embeddings,
    text_embeddings,
    temperature=TEMPERATURE,
    apply_softmax='over_texts',
)

In [7]:
v2t_similarity_vector = similarity_matrix[0]
top_indices = np.argsort(v2t_similarity_vector)[::-1]

print(f'Query video: {os.path.basename(VIDEO_FILE_PATH)}')
mediapy.show_video(frames[0], fps=6.0)

for k, j in enumerate(top_indices):
  print(
      'Top-%d retrieved text: %s [Similarity = %0.4f]'
      % (k + 1, text_queries[j], v2t_similarity_vector[j])
  )
print(f'\nThis is {text_queries[top_indices[0]]}')

Query video: water_bottle_drumming.mp4


This browser does not support the video tag.


Top-1 retrieved text: a video of sitting. [Similarity = 0.6414]
Top-2 retrieved text: a video of playing drums. [Similarity = 0.2726]
Top-3 retrieved text: a video of playing flute. [Similarity = 0.0442]
Top-4 retrieved text: a video of playing at playground. [Similarity = 0.0409]
Top-5 retrieved text: a video of concert. [Similarity = 0.0009]

This is a video of sitting.


In [12]:
!pip install wget openpyxl opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.7 MB/s eta 0:00:00


In [9]:
import pandas as pd

In [13]:
data_file = '/content/video_rating_results.xlsx'
df = pd.read_excel(data_file)
df.loc[:, 'md5']
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   caption                             53 non-null     object 
 1   video_url                           53 non-null     object 
 2   sa                                  53 non-null     int64  
 3   pc                                  53 non-null     int64  
 4   joint                               53 non-null     int64  
 5   action                              52 non-null     object 
 6   is_hard                             53 non-null     int64  
 7   physics_rules_followed              53 non-null     object 
 8   physics_rules_unfollowed            53 non-null     object 
 9   physics_rules_cannot_be_determined  53 non-null     object 
 10  human_violated_rules                53 non-null     object 
 11  model_name                          53 non-null

,caption,video_url,sa,pc,joint,action,is_hard,physics_rules_followed,physics_rules_unfollowed,physics_rules_cannot_be_determined,...,prompt_sa,prompt_physics,prompt_sa_score,sa_gemini_2.5_flash,prompt_sa_usage,prompt_physics_score,pc_gemini_2.5_flash,prompt_physics_usage,prompt_sa_cost,prompt_physics_cost
0,"A hedge trimmer is used to shape a tall, dense...",https://videophysics2testvideos.s3.us-east-2.a...,4,3,0,trimming shrubs,0,"['The blades of the hedge trimmer move.', 'The...",['The cut pieces of the hedge fall to the grou...,[],...,The following is a conversation between a curi...,The following is a conversation between a curi...,This video is an excellent match for the descr...,5,"{""completion_tokens"":197,""prompt_tokens"":1415,...",This video appears to fully adhere to the phys...,5,"{""completion_tokens"":68,""prompt_tokens"":1388,""...",0.003726,0.003554
1,A leaf blower is pointed at a patch of leaves ...,https://videophysics2testvideos.s3.us-east-2.a...,4,4,1,blowing leaves,0,"['The leaves move away from the leaf blower.',...",[],[],...,The following is a conversation between a curi...,The following is a conversation between a curi...,This video is an excellent match for the descr...,5,"{""completion_tokens"":141,""prompt_tokens"":1151,...",This video appears to adhere very well to the ...,5,"{""completion_tokens"":269,""prompt_tokens"":1125,...",0.003022,0.003001
2,"A seamstress pulls the fabric taut, guiding th...",https://videophysics2testvideos.s3.us-east-2.a...,4,4,1,sewing,0,['The needle moves up and down in a repetitive...,[],[],...,The following is a conversation between a curi...,The following is a conversation between a curi...,This video is an excellent match for the descr...,5,"{""completion_tokens"":177,""prompt_tokens"":885,""...",This video depicts a common and realistic acti...,5,"{""completion_tokens"":282,""prompt_tokens"":862,""...",0.002379,0.002370
3,"A seamstress pulls the fabric taut, guiding th...",https://videophysics2testvideos.s3.us-east-2.a...,4,5,1,sewing,0,"[""The woman's headscarf and shirt remain in a ...",[],[],...,The following is a conversation between a curi...,The following is a conversation between a curi...,"The video shows a woman handling red fabric, w...",2,"{""completion_tokens"":220,""prompt_tokens"":1411,...","Based on the video provided, there are no appa...",5,"{""completion_tokens"":75,""prompt_tokens"":1388,""...",0.003857,0.003541
4,"Hands fold a map, showing the creases forming ...",https://videophysics2testvideos.s3.us-east-2.a...,1,4,0,folding paper,0,['The hands maintain a relatively static posit...,[],[],...,The following is a conversation between a curi...,The following is a conversation between a curi...,"Based on the video provided, the description ""...",1,"{""completion_tokens"":102,""prompt_tokens"":879,""...","Based on the video provided, which shows hands...",5,"{""completion_tokens"":78,""prompt_tokens"":862,""t...",0.002333,0.002239


In [14]:
# @title Download videos

# Create a directory to save the videos
if not os.path.exists('videos'):
    os.makedirs('videos')

# Iterate through the DataFrame and download each video
for index, row in df.iterrows():
    video_url = row['video_url']
    md5 = row['md5']
    output_filename = f'videos/{md5}.mp4'  # Assuming the videos are mp4 format

    # Check if the file already exists to avoid re-downloading
    if not os.path.exists(output_filename):
        print(f"Downloading {video_url} to {output_filename}...")
        !wget -O {output_filename} {video_url}
    else:
        print(f"File {output_filename} already exists, skipping download.")

--2025-07-31 07:16:14--  https://videophysics2testvideos.s3.us-east-2.amazonaws.com/cosmos_videophy2_test_challenging/A_hedge_trimmer_is_used_to_shape_a_tall,_dense_privet_hedge,_the_blades_visibly_cutting_through_the_leaves_and_stems.mp4
Resolving videophysics2testvideos.s3.us-east-2.amazonaws.com (videophysics2testvideos.s3.us-east-2.amazonaws.com)... 52.219.98.98, 52.219.228.122, 3.5.130.145, ...
Connecting to videophysics2testvideos.s3.us-east-2.amazonaws.com (videophysics2testvideos.s3.us-east-2.amazonaws.com)|52.219.98.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1279915 (1.2M) [video/mp4]
Saving to: ‘videos/d667e27fee2f6a7a67aa6047b7708d03.mp4’

videos/d667e27fee2f 100%[===================>]   1.22M  --.-KB/s    in 0.1s    

2025-07-31 07:16:14 (10.9 MB/s) - ‘videos/d667e27fee2f6a7a67aa6047b7708d03.mp4’ saved [1279915/1279915]

--2025-07-31 07:16:14--  https://videophysics2testvideos.s3.us-east-2.amazonaws.com/hunyuan_xdit/A_leaf_blower_is_pointed

In [43]:
import cv2

def read_and_preprocess_video(
    filename: str,
    target_num_frames: int,
    target_frame_size: tuple[int, int]  # 保持原有参数签名
):
    """读取并预处理视频，自动转换为正方形帧"""
    frames = mediapy.read_video(filename)

    # 采样目标帧数
    frame_indices = np.linspace(
        0, len(frames), num=target_num_frames, endpoint=False, dtype=np.int32
    )
    frames = np.array([frames[i] for i in frame_indices])

    # 检查是否已经是正方形
    target_height, target_width = target_frame_size
    if target_height != target_width:
        print(f"警告: 目标尺寸 {target_frame_size} 不是正方形。强制转换为正方形 ({target_height}x{target_height})")
        target_frame_size = (target_height, target_height)  # 使用高度创建正方形

    # 转换为正方形处理
    processed_frames = []
    for frame in frames:
        # 1. 保持原始宽高比缩放
        h, w = frame.shape[:2]
        scale = target_height / max(h, w)
        new_h, new_w = int(h * scale), int(w * scale)
        resized = cv2.resize(frame, (new_w, new_h))

        # 2. 创建正方形画布
        canvas = np.zeros((target_height, target_height, 3), dtype=np.uint8)

        # 3. 居中放置缩放后的图像
        y_offset = (target_height - new_h) // 2
        x_offset = (target_height - new_w) // 2
        canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

        processed_frames.append(canvas)

    frames = np.array(processed_frames)

    # 归一化像素值 [0, 1]
    frames = mediapy.to_float01(frames)

    return frames



In [53]:
# 物理原则二分类查询
text_queries = [
    'a video showing physically impossible scenarios with violations of fundamental physics principles',
    'a video showing realistic physics and natural motion following conservation laws'
]

text_ids, text_paddings = vp.tokenize_texts(text_tokenizer, text_queries)


# 视频转换
idx = 5
NUM_FRAMES = 16
# 352附近最近的18的倍数
closest_size = 18 * round(352 / 18)  # 18 * 20 = 360

# 使用调整后的尺寸
height = width = closest_size  # 360x360
VIDEO_FILE_PATH = './videos/' + df.loc[idx, 'md5'] + '.mp4'


frames = read_and_preprocess_video(
    VIDEO_FILE_PATH,
    target_num_frames=NUM_FRAMES,
    target_frame_size=[height, width],
)
frames = jnp.asarray(frames[None, ...])  # Add batch dimension.

# @title Compute video-to-text retrieval results
video_embeddings, text_embeddings, _ = forward_fn(
    frames, text_ids, text_paddings)

TEMPERATURE = 0.01  # @param {type: "number"}
similarity_matrix = compute_similarity_matrix(
    video_embeddings,
    text_embeddings,
    temperature=TEMPERATURE,
    apply_softmax='over_texts',
)

v2t_similarity_vector = similarity_matrix[0]
top_indices = np.argsort(v2t_similarity_vector)[::-1]

print(f'Query video: {os.path.basename(VIDEO_FILE_PATH)}')
mediapy.show_video(frames[0], fps=6.0)

for k, j in enumerate(top_indices):
  print(
      'Top-%d retrieved text: %s [Similarity = %0.4f]'
      % (k + 1, text_queries[j], v2t_similarity_vector[j])
  )
print(f'\nThis is {text_queries[top_indices[0]]}')

Query video: 0239405236100bb9e54700a9ad85abdd.mp4


This browser does not support the video tag.


Top-1 retrieved text: a video showing physically impossible scenarios with violations of fundamental physics principles [Similarity = 0.5467]
Top-2 retrieved text: a video showing realistic physics and natural motion following conservation laws [Similarity = 0.4533]

This is a video showing physically impossible scenarios with violations of fundamental physics principles


In [55]:
def analyze_video_physics(video_path, target_size=352, num_frames=16, temperature=0.01, show_video=True):
    """
    分析视频是否符合物理原理

    Args:
        video_path: 视频文件路径
        target_size: 目标帧尺寸
        num_frames: 目标帧数
        temperature: 相似度计算温度参数
        show_video: 是否显示视频

    Returns:
        dict: 包含分析结果的字典
    """
    # 物理原则二分类查询
    text_queries = [
        'a video showing physically impossible scenarios with violations of fundamental physics principles',
        'a video showing realistic physics and natural motion following conservation laws'
    ]

    text_ids, text_paddings = vp.tokenize_texts(text_tokenizer, text_queries)

    # 计算最接近的18的倍数尺寸
    closest_size = 18 * round(target_size / 18)
    height = width = closest_size

    # 视频预处理
    frames = read_and_preprocess_video(
        video_path,
        target_num_frames=num_frames,
        target_frame_size=[height, width],
    )
    frames = jnp.asarray(frames[None, ...])  # Add batch dimension

    # 计算视频到文本的检索结果
    video_embeddings, text_embeddings, _ = forward_fn(
        frames, text_ids, text_paddings)

    similarity_matrix = compute_similarity_matrix(
        video_embeddings,
        text_embeddings,
        temperature=temperature,
        apply_softmax='over_texts',
    )

    v2t_similarity_vector = similarity_matrix[0]
    top_indices = np.argsort(v2t_similarity_vector)[::-1]

    # 输出结果
    print(f'Query video: {os.path.basename(video_path)}')

    if show_video:
        mediapy.show_video(frames[0], fps=6.0)

    for k, j in enumerate(top_indices):
        print(
            'Top-%d retrieved text: %s [Similarity = %0.4f]'
            % (k + 1, text_queries[j], v2t_similarity_vector[j])
        )

    result_text = text_queries[top_indices[0]]
    print(f'\nThis is {result_text}')

    # 返回结果
    return {
        'video_path': video_path,
        'similarity_scores': v2t_similarity_vector,
        'top_indices': top_indices,
        'text_queries': text_queries,
        'prediction': result_text,
        'is_physically_realistic': top_indices[0] == 1  # 第二个查询是现实物理
    }

# 使用示例：
idx = 0
file_path = './videos/' + df.loc[idx, 'md5'] + '.mp4'
result = analyze_video_physics(file_path)
result

Query video: d667e27fee2f6a7a67aa6047b7708d03.mp4


This browser does not support the video tag.


Top-1 retrieved text: a video showing realistic physics and natural motion following conservation laws [Similarity = 0.5929]
Top-2 retrieved text: a video showing physically impossible scenarios with violations of fundamental physics principles [Similarity = 0.4071]

This is a video showing realistic physics and natural motion following conservation laws


{'video_path': './videos/d667e27fee2f6a7a67aa6047b7708d03.mp4',
 'similarity_scores': array([0.40711015, 0.59288985], dtype=float32),
 'top_indices': array([1, 0]),
 'text_queries': ['a video showing physically impossible scenarios with violations of fundamental physics principles',
  'a video showing realistic physics and natural motion following conservation laws'],
 'prediction': 'a video showing realistic physics and natural motion following conservation laws',
 'is_physically_realistic': np.True_}

In [58]:
from tqdm.auto import tqdm
tqdm.pandas(desc='inferencing...')

for idx, row in tqdm(df.iterrows(), total=len(df)):
  try:
    file_path = './videos/' + row['md5'] + '.mp4'
    result = analyze_video_physics(file_path, show_video=False)
    df.loc[idx, 'prediction'] = result
  except Exception as e:
    print(row ,e)
    df.loc[idx, 'prediction'] = 'Error'

  0%|          | 0/53 [00:00<?, ?it/s]

Query video: d667e27fee2f6a7a67aa6047b7708d03.mp4
Top-1 retrieved text: a video showing realistic physics and natural motion following conservation laws [Similarity = 0.5929]
Top-2 retrieved text: a video showing physically impossible scenarios with violations of fundamental physics principles [Similarity = 0.4071]

This is a video showing realistic physics and natural motion following conservation laws
caption                               A leaf blower is pointed at a patch of leaves ...
video_url                             https://videophysics2testvideos.s3.us-east-2.a...
sa                                                                                    4
pc                                                                                    4
joint                                                                                 1
action                                                                   blowing leaves
is_hard                                                          

In [60]:
df.prediction.values[0]

'./videos/d667e27fee2f6a7a67aa6047b7708d03.mp4'